Comments with double pound sign (##) were made by me in Jupyter. All other prompts were used for code generation.

In [44]:
#python
#using pyomo define the variables and parameters needed for a simple two stage unit commitment problem

#import pyomo
from pyomo.environ import *

#create a model object
model = ConcreteModel()

# define generators
generator_list = [
    {'startup_cost': 50, 'production_cost': 100, 'max_output': 200, 'min_output': 0},
    {'startup_cost': 30, 'production_cost': 80, 'max_output': 150, 'min_output': 50},
    {'startup_cost': 20, 'production_cost': 70, 'max_output': 100, 'min_output': 0},
    {'startup_cost': 10, 'production_cost': 40, 'max_output': 70, 'min_output': 0},
    {'startup_cost': 5, 'production_cost': 20, 'max_output': 30, 'min_output': 0}
]

# define sets
model.stages = Set(initialize = [1,2])
model.units = Set(initialize = [1,2,3,4,5])
demand = {1:300,2:500} #this is also adjusted before solving!
load_shed_cost = 500

In [45]:
# define parameters
model.demand = Param(model.stages, within=NonNegativeReals, initialize = demand, mutable = True)
model.load_shed_cost = Param( within=NonNegativeReals, initialize = load_shed_cost, mutable = True)

# add generator data to the model
model.startup_costs = Param(model.units, initialize=dict(zip(model.units, [g['startup_cost'] for g in generator_list])))
model.production_costs = Param(model.units, initialize=dict(zip(model.units, [g['production_cost']  for g in generator_list])))

#define generator parameters and initialize them with generator_list
model.max_output = Param(model.units, initialize=dict(zip(model.units, [g['max_output'] for g in generator_list])))
model.min_output = Param(model.units, initialize=dict(zip(model.units, [g['min_output'] for g in generator_list])))

# define variables
model.commit = Var(model.stages, model.units, within=Binary)
model.onoff = Var(model.stages, model.units, within=Binary)
model.production = Var(model.stages, model.units, within=NonNegativeReals)
model.load_shed = Var(model.stages, within=NonNegativeReals)

In [46]:
#define the unit commitment constraints and cost function

#commitment constraints
def commit_rule(model,s,u):
    return model.commit[s,u] <= model.onoff[s,u]
model.commit_constraint = Constraint(model.stages, model.units, rule=commit_rule)

#onoff constraints
def onoff_rule(model,s,u):
    if s == 1:
        return model.onoff[s,u] == model.commit[s,u]
    elif s == 2:
        return model.onoff[s,u] == model.commit[s,u] + model.onoff[s-1,u]
model.onoff_constraint = Constraint(model.stages, model.units, rule=onoff_rule)

#min and max output constraints
def output_rule(model,s,u):
    return model.production[s,u] <= model.onoff[s,u]*model.max_output[u]
model.output_constraint = Constraint(model.stages, model.units, rule=output_rule)

#demand constraint
def demand_rule(model,s):
    return sum(model.production[s,u] for u in model.units) + model.load_shed[s] == model.demand[s]
model.demand_constraint = Constraint(model.stages, rule=demand_rule)
    
#cost function
def cost_func(model):
    return sum(sum(model.startup_costs[u]*model.commit[s,u] + model.production_costs[u] * model.production[s,u] for u in model.units) + model.load_shed_cost * model.load_shed[s] for s in model.stages)
model.cost = Objective(rule=cost_func, sense=minimize)

In [47]:
## change the demand
demand = [100,300]
for i, d in enumerate(demand):   
    model.demand[i+1] = d
    
#solve model and print the results    
solver = SolverFactory('gurobi')
solver.solve(model)

##Total Cost
print('Total Cost: ', value(model.cost))
print('Load Shed Cost: ', sum(value(model.load_shed_cost)*value(model.load_shed[s]) for s in model.stages))

#print the results for both timesteps
#for time step 1
print("Time step 1:")
for u in model.units:
    print("Unit " + str(u) + " Commitment: " + str(model.commit[1,u]()) + " Production: " + str(model.production[1,u]()))

#for time step 2
print("Time step 2:")
for u in model.units:
    print("Unit " + str(u) + " Commitment: " + str(model.commit[2,u]()) + " Production: " + str(model.production[2,u]()))

Total Cost:  21865.0
Load Shed Cost:  0.0
Time step 1:
Unit 1 Commitment: 0.0 Production: 0.0
Unit 2 Commitment: 1.0 Production: 0.0
Unit 3 Commitment: 1.0 Production: 0.0
Unit 4 Commitment: 1.0 Production: 70.0
Unit 5 Commitment: 1.0 Production: 30.0
Time step 2:
Unit 1 Commitment: 0.0 Production: 0.0
Unit 2 Commitment: 0.0 Production: 100.0
Unit 3 Commitment: 0.0 Production: 100.0
Unit 4 Commitment: 0.0 Production: 70.0
Unit 5 Commitment: 0.0 Production: 30.0
